## Imports 

In [1]:
from SETUP import *
from REDUCE_FEATURES import *
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
data_file = '/Users/Winnifred/Desktop/Capstone/ICPSR_20240_RAWDATA/DS0001/20240-0001-Data.tsv'
filenames = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_group_file_names.txt'
csv_root_path = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_name_data/'

In [3]:
set_inst = Setup(csv_root_path, filenames)
full_dict = set_inst.execute_setup()
reduce_inst = Reduce_Features(data_file, full_dict)
dirty_df = reduce_inst.execute_reduce()

In [4]:
df = dirty_df.drop(dirty_df[dirty_df['V01993'] == ' '].index)

In [5]:
## Turn strings to ints 
mask = {str(num): num for num in range(200000)}

In [6]:
mask[' '] = 0

In [7]:
mask['-9'] = -9

In [8]:
mask['-8'] = -8

In [9]:
mask[None] = 0

In [10]:
for idx, feature in enumerate(list(df)): 
    df[feature] = df[feature].map(mask)
    if idx % 250 == 0: 
        print('completed feature {} of 2110'.format(idx))

completed feature 0 of 2110
completed feature 250 of 2110
completed feature 500 of 2110
completed feature 750 of 2110
completed feature 1000 of 2110


In [11]:
df.head()

,V01638,V01639,V01643,V01644,V01646,V01647,V01648,V01649,V01650,V01651,...,V08495,V08549,V07725,V07894,V08501,V08500,V08553,V07750,V07748,V07899
0,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,0,0,0,0,0,0,0,0,0,0,...,5,5,1,1,5,5,5,5,5,5
3,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
5,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
6,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5


In [12]:
df.isnull().any()

V01638    False
V01639    False
V01643    False
V01644    False
V01646    False
V01647    False
V01648    False
V01649    False
V01650    False
V01651    False
V01652    False
V01653    False
V01657    False
V01658    False
V01659    False
V01660    False
V01661    False
V01662    False
V01663    False
V01664    False
V01667    False
V01668    False
V01669    False
V01670    False
V01671    False
V01672    False
V01673    False
V01674    False
V01675    False
V01676    False
          ...  
V08403    False
V08534    False
V08415    False
V08414    False
V08535    False
V07661    False
V07659    False
V07877    False
V08477    False
V08476    False
V08546    False
V07701    False
V07699    False
V07883    False
V08494    False
V08493    False
V08548    False
V07718    False
V07716    False
V07888    False
V08495    False
V08549    False
V07725    False
V07894    False
V08501    False
V08500    False
V08553    False
V07750    False
V07748    False
V07899    False
Length: 1217, dtype: boo

In [13]:
df = df.dropna(axis=1)

In [14]:
df['V01993'].describe()

count    15890.000000
mean         4.456765
std          1.429194
min         -9.000000
25%          5.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: V01993, dtype: float64

# Step 3  
## Logistic Regression

In [15]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

In [16]:
X_df = df.loc[:, df.columns != 'V01993']
y_df = df['V01993'].map({5:0, 1:1, -9:0, -8:0})

In [17]:
# y_df.map({5:0, 1:1, -9:0, -8:0})

In [18]:
X = X_df.as_matrix()
y = y_df.as_matrix()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(11123, 1208) (11123,)
(4767, 1208) (4767,)


In [21]:
logreg = linear_model.LogisticRegression()

In [22]:
model = logreg.fit(X_train, y_train)

In [23]:
y_pred = logreg.predict(X_test)

In [24]:
print('Score:', model.score(X_test, y_test))

Score: 0.999160897839


In [25]:
def print_scores(y_test, predictions):
    print('Accuracy:', accuracy_score(y_test, predictions))
    print('Precision:', precision_score(y_test, predictions))
    print('Recall:', recall_score(y_test, predictions))
    return None

In [26]:
print_scores(y_test, y_pred)

Accuracy: 0.999160897839
Precision: 1.0
Recall: 0.993333333333


In [27]:
cnf_matrix = confusion_matrix(y, y)

In [28]:
cnf_matrix

array([[13800,     0],
       [    0,  2090]])

In [29]:
1156/9757

0.1184790406887363

In [30]:
8601/9757

0.8815209593112637

## Linear Regression

In [31]:
from sklearn.metrics import mean_squared_error, r2_score


In [32]:
regr = linear_model.LinearRegression()

In [33]:
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
y_pred = regr.predict(X_test)

In [35]:
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [-0.00322477 -0.00156831  0.00057822 ...,  0.00218514 -0.00201927
  0.00025596]


In [36]:
print("Mean squared error: {}".format(mean_squared_error(y_test, y_pred)))

Mean squared error: 2534428781447460.5


In [37]:
print('Variance score: {}'.format(r2_score(y_test, y_pred)))

Variance score: -2.3035394000291944e+16


## Random Forest

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [40]:
clf = RandomForestClassifier(max_depth=20, random_state=0)

In [41]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [42]:
print(clf.feature_importances_)

[ 0.          0.00042774  0.         ...,  0.          0.00012356  0.        ]


In [43]:
y_pred = clf.predict(X_test)

In [44]:
print_scores(y_test, y_pred)

Accuracy: 0.998112020138
Precision: 1.0
Recall: 0.985
